- mudar o length de products que está errado
- verificar as colunas de relação, que não podem ter null

pip:
- notebook
- pandas

In [ ]:
!curl -L -o brazilian-ecommerce.zip https://www.kaggle.com/api/v1/datasets/download/olistbr/brazilian-ecommerce
!mkdir brazilian_ecommerce
!unzip brazilian-ecommerce.zip -d brazilian_ecommerce
!rm brazilian-ecommerce.zip

In [ ]:
!pip install pandas

In [1]:
# importação de bibliotecas

import os
import pandas as pd

In [2]:
folder = "brazilian_ecommerce"

# Lista todos os arquivos CSV na pasta
csv_files = [f for f in os.listdir(folder) if f.endswith('.csv')]
dataframe_names = []

# Carrega os DataFrames dinamicamente
for file in csv_files:
    # Remove extensão .csv
    base_name = file.replace('.csv', '')
    # Remove 'olist_', troca 'dataset' por 'df'
    df_name = base_name.replace('olist_', '').replace('dataset', 'df')
    # Lê o CSV
    complete_path = os.path.join(folder, file)
    dataframe_names.append(df_name)
    globals()[df_name] = pd.read_csv(complete_path)
    print(f"{df_name} loaded with {globals()[df_name].shape[0]} lines.")

sellers_df loaded with 3095 lines.
product_category_name_translation loaded with 71 lines.
orders_df loaded with 99441 lines.
order_items_df loaded with 112650 lines.
customers_df loaded with 99441 lines.
geolocation_df loaded with 1000163 lines.
order_payments_df loaded with 103886 lines.
order_reviews_df loaded with 99224 lines.
products_df loaded with 32951 lines.


In [17]:
# analisando dataframes
for df_name in dataframe_names:
    print(f"DataFrame: {df_name}")
    display(globals()[df_name].head(3))
    print()

DataFrame: sellers_df


,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ



DataFrame: product_category_name_translation


,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto



DataFrame: orders_df


...

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71



DataFrame: order_reviews_df


,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24



DataFrame: products_df


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0


Ao analisar o nome das colunas, é possível perceber que no dataframe **products_df** existem duas colunas com o nome errado:
- product_name_lenght
- product_description_lenght

Como o correto é **length**, estarei corrigindo o nome.

## 🧼 Análise de Colunas com Possíveis Valores Nulos

### ✅ Objetivo
Identificar quais colunas de cada DataFrame do dataset da Olist **podem conter valores nulos**, com base no tipo da coluna e na função dela (chave primária, chave estrangeira ou campo informativo).

---

### 📌 Regras Gerais

| Tipo de Chave        | Pode ter `null`? | Deve ser Única? |
|----------------------|------------------|------------------|
| **Chave Primária**   | ❌ Nunca          | ✅ Sim           |
| **Chave Estrangeira**| ✅ Sim (às vezes) | ❌ Não           |

---

## 🔍 Análise Dataset por Dataset

### `sellers_df`

| Coluna                  | Pode ter `null`? | Justificativa                     |
|------------------------|------------------|-----------------------------------|
| `seller_id`            | ❌ Não            | Chave primária                    |
| `seller_zip_code_prefix` | ❌ Não          | Localização obrigatória           |
| `seller_city`          | ❌ Não            | Sempre presente                   |
| `seller_state`         | ❌ Não            | Sempre presente                   |

---

### `product_category_name_translation`

| Coluna                          | Pode ter `null`? | Justificativa                     |
|--------------------------------|------------------|-----------------------------------|
| `product_category_name`        | ❌ Não            | Chave primária                    |
| `product_category_name_english`| ❌ Não            | Tradução correspondente           |

---

### `orders_df`

| Coluna                          | Pode ter `null`? | Justificativa                         |
|---------------------------------|------------------|---------------------------------------|
| `order_id`                      | ❌ Não            | Chave primária                        |
| `customer_id`                   | ❌ Não            | FK obrigatória                        |
| `order_status`                  | ❌ Não            | Status sempre presente                |
| `order_purchase_timestamp`      | ❌ Não            | Sempre presente                       |
| `order_approved_at`             | ✅ Sim            | Pode não ter sido aprovado ainda      |
| `order_delivered_carrier_date`  | ✅ Sim            | Pode não ter sido enviado             |
| `order_delivered_customer_date` | ✅ Sim            | Pode não ter sido entregue            |
| `order_estimated_delivery_date` | ❌ Não            | Sempre gerada                         |

---

### `order_items_df`

| Coluna               | Pode ter `null`? | Justificativa                                    |
|----------------------|------------------|--------------------------------------------------|
| `order_id`           | ❌ Não            | Chave estrangeira                                |
| `order_item_id`      | ❌ Não            | Parte da chave primária composta                 |
| `product_id`         | ✅ Sim (raro)     | Pode haver produto não cadastrado                |
| `seller_id`          | ❌ Não            | Sempre presente                                  |
| `shipping_limit_date`| ❌ Não            | Data definida para envio                         |
| `price`              | ❌ Não            | Sempre presente                                  |
| `freight_value`      | ❌ Não            | Sempre presente                                  |

---

### `customers_df`

| Coluna                     | Pode ter `null`? | Justificativa                  |
|----------------------------|------------------|--------------------------------|
| `customer_id`              | ❌ Não            | Chave primária                |
| `customer_unique_id`       | ❌ Não            | Sempre presente               |
| `customer_zip_code_prefix` | ❌ Não            | Localização                   |
| `customer_city`            | ❌ Não            | Sempre presente               |
| `customer_state`           | ❌ Não            | Sempre presente               |

---

### `geolocation_df`

| Coluna                        | Pode ter `null`? | Justificativa                          |
|-------------------------------|------------------|----------------------------------------|
| `geolocation_zip_code_prefix` | ❌ Não            | Usado como referência de localização   |
| `geolocation_lat`             | ❌ Não            | Latitude                                |
| `geolocation_lng`             | ❌ Não            | Longitude                               |
| `geolocation_city`            | ❌ Não            | Sempre presente                         |
| `geolocation_state`           | ❌ Não            | Sempre presente                         |

> ⚠️ Apesar de não ter chave primária formal, essa tabela não deve conter `nulls`.

---

### `order_payments_df`

| Coluna               | Pode ter `null`? | Justificativa              |
|----------------------|------------------|----------------------------|
| `order_id`           | ❌ Não            | Chave estrangeira         |
| `payment_sequential` | ❌ Não            | Parte da chave composta   |
| `payment_type`       | ❌ Não            | Sempre presente           |
| `payment_installments`| ❌ Não           | Sempre presente           |
| `payment_value`      | ❌ Não            | Sempre presente           |

---

### `order_reviews_df`

| Coluna                   | Pode ter `null`? | Justificativa                               |
|---------------------------|------------------|---------------------------------------------|
| `review_id`              | ❌ Não            | Chave primária                              |
| `order_id`               | ❌ Não            | FK obrigatória                              |
| `review_score`           | ❌ Não            | Sempre presente                             |
| `review_comment_title`   | ✅ Sim            | Comentário opcional                         |
| `review_comment_message` | ✅ Sim            | Comentário opcional                         |
| `review_creation_date`   | ❌ Não            | Sempre presente                             |
| `review_answer_timestamp`| ✅ Sim            | Pode não ter sido respondido ainda          |

---

### `products_df`

| Coluna                         | Pode ter `null`? | Justificativa                                  |
|--------------------------------|------------------|------------------------------------------------|
| `product_id`                   | ❌ Não            | Chave primária                                |
| `product_category_name`        | ✅ Sim            | Pode ser desconhecida                          |
| `product_name_lenght`          | ✅ Sim            | Dados ausentes para alguns produtos            |
| `product_description_lenght`   | ✅ Sim            | Idem acima                                     |
| `product_photos_qty`           | ✅ Sim            | Idem                                           |
| `product_weight_g`             | ✅ Sim            | Pode estar ausente                             |
| `product_length_cm`            | ✅ Sim            | Idem                                           |
| `product_height_cm`            | ✅ Sim            | Idem                                           |
| `product_width_cm`             | ✅ Sim            | Idem                                           |

---

## ✅ Conclusão

- As únicas colunas que **devem obrigatoriamente ser preenchidas** são **chaves primárias** e campos essenciais.
- Vários campos podem conter `null`, especialmente:
  - **Datas de aprovação/entrega** (`orders_df`)
  - **Campos de comentário** (`order_reviews_df`)
  - **Atributos de produto** (`products_df`)
- Esses `nulls` devem ser **tratados com atenção** dependendo da análise (remoção, imputação, separação, etc).

---


In [16]:
# Dicionário com colunas que não podem ter null para cada df
nao_pode_ter_null = {
    'orders_df': ['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_estimated_delivery_date'],
    'order_items_df': ['order_id', 'order_item_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value'],
    'customers_df': ['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state'],
    'geolocation_df': ['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state'],
    'order_payments_df': ['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value'],
    'order_reviews_df': ['review_id', 'order_id', 'review_score', 'review_creation_date'],
    'products_df': ['product_id'],
    'sellers_df': ['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state'],
    'product_category_name_translation': ['product_category_name', 'product_category_name_english']
}

# Limpeza automática
for df_name, colunas in nao_pode_ter_null.items():
    df = globals()[df_name]
    antes = df.shape[0]
    df.dropna(subset=colunas, inplace=True)
    depois = df.shape[0]
    print(f"{df_name}: {antes - depois} linhas removidas devido a valores nulos nas colunas obrigatórias.")


orders_df: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.
order_items_df: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.
customers_df: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.
geolocation_df: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.
order_payments_df: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.
order_reviews_df: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.
products_df: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.
sellers_df: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.
product_category_name_translation: 0 linhas removidas devido a valores nulos nas colunas obrigatórias.


In [15]:
# analisando dataframes
for df_name in dataframe_names:
    print(f"DataFrame: {df_name}")
    display(globals()[df_name].columns)
    print()

DataFrame: sellers_df


Index(['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state'], dtype='object')


DataFrame: product_category_name_translation


Index(['product_category_name', 'product_category_name_english'], dtype='object')


DataFrame: orders_df


...

Index(['order_id', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value'],
      dtype='object')


DataFrame: order_reviews_df


Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')


DataFrame: products_df


Index(['product_id', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm'],
      dtype='object')

In [5]:
# verificando tipos
for df_name in dataframe_names:
    print(df_name)
    df = globals()[df_name]
    print(df.dtypes)
    print()

sellers_df
seller_id                 object
seller_zip_code_prefix     int64
seller_city               object
seller_state              object
dtype: object

product_category_name_translation
product_category_name            object
product_category_name_english    object
dtype: object

orders_df
order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

order_items_df
order_id                object
order_item_id            int64
product_id              object
seller_id               object
shipping_limit_date     object
price                  float64
freight_value          float64
dtype: object

customers_df
customer_id                 object
customer_unique_id          object
customer_zip_code_prefix     int64
cus

In [4]:
# Verificando duplicatas

for df_name in dataframe_names:
    print(df_name)
    df = globals()[df_name]
    print("Pedidos duplicados:", df.duplicated().sum())

sellers_df
Pedidos duplicados: 0
product_category_name_translation
Pedidos duplicados: 0
orders_df
Pedidos duplicados: 0
order_items_df
Pedidos duplicados: 0
customers_df
Pedidos duplicados: 0
geolocation_df
Pedidos duplicados: 261831
order_payments_df
Pedidos duplicados: 0
order_reviews_df
Pedidos duplicados: 0
products_df
Pedidos duplicados: 0


In [5]:
# geolocation é o único com duplicatas
geolocation_df = geolocation_df.drop_duplicates()
geolocation_df.duplicated().sum()

np.int64(0)

In [6]:
# Verificando duplicatas novamente

for df_name in dataframe_names:
    print(df_name)
    df = globals()[df_name]
    print("Pedidos duplicados:", df.duplicated().sum())

sellers_df
Pedidos duplicados: 0
product_category_name_translation
Pedidos duplicados: 0
orders_df
Pedidos duplicados: 0
order_items_df
Pedidos duplicados: 0
customers_df
Pedidos duplicados: 0
geolocation_df
Pedidos duplicados: 0
order_payments_df
Pedidos duplicados: 0
order_reviews_df
Pedidos duplicados: 0
products_df
Pedidos duplicados: 0


In [ ]:
customers_df = customers_df.astype({
    'customer_id': 'string',
    'customer_unique_id': 'string',
    'customer_zip_code_prefix': 'string',  # ZIP code tratado como texto
    'customer_city': 'string',
    'customer_state': 'string'
})

geolocation_df = geolocation_df.astype({
    'geolocation_zip_code_prefix': 'string',  # ZIP tratado como texto
    'geolocation_lat': 'float64',             # Latitude
    'geolocation_lng': 'float64',             # Longitude
    'geolocation_city': 'string',             # Nome da cidade
    'geolocation_state': 'string'             # Sigla do estado
})

order_items_df = order_items_df.astype({
    'order_id': 'string',
    'order_item_id': 'int64',
    'product_id': 'string',
    'seller_id': 'string',
    'price': 'float64',
    'freight_value': 'float64'
})

order_items_df['shipping_limit_date'] = pd.to_datetime(order_items_df['shipping_limit_date'])

order_payments_df = order_payments_df.astype({
    'order_id': 'string',
    'payment_sequential': 'int64',
    'payment_type': 'string',
    'payment_installments': 'int64',
    'payment_value': 'float64'
})

order_reviews_df = order_reviews_df.astype({
    'review_id': 'string',
    'order_id': 'string',
    'review_score': 'int64',
    'review_comment_title': 'string',
    'review_comment_message': 'string'
})

order_reviews_df['review_creation_date'] = pd.to_datetime(order_reviews_df['review_creation_date'])
order_reviews_df['review_answer_timestamp'] = pd.to_datetime(order_reviews_df['review_answer_timestamp'])

orders_df = orders_df.astype({
    'order_id': 'string',
    'customer_id': 'string',
    'order_status': 'string'
})

orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])
orders_df['order_approved_at'] = pd.to_datetime(orders_df['order_approved_at'])
orders_df['order_delivered_carrier_date'] = pd.to_datetime(orders_df['order_delivered_carrier_date'])
orders_df['order_delivered_customer_date'] = pd.to_datetime(orders_df['order_delivered_customer_date'])
orders_df['order_estimated_delivery_date'] = pd.to_datetime(orders_df['order_estimated_delivery_date'])

products_df = products_df.astype({
    'product_id': 'string',
    'product_category_name': 'string',
    'product_name_lenght': 'int64',
    'product_description_lenght': 'int64',
    'product_photos_qty': 'int64',
    'product_weight_g': 'float64',
    'product_length_cm': 'float64',
    'product_height_cm': 'float64',
    'product_width_cm': 'float64'
})

sellers_df = sellers_df.astype({
    'seller_id': 'string',
    'seller_zip_code_prefix': 'string',  # Trata o prefixo do CEP como string para manter o formato correto
    'seller_city': 'string',
    'seller_state': 'string'
})

product_category_df = product_category_df.astype({
    'product_category_name': 'string',
    'product_category_name_english': 'string'
})


In [ ]:
nulls_in_product_name_lenght = products_df['product_name_lenght'].isnull()

products_df[nulls_in_product_name_lenght]

In [ ]:
nulos = products_df['product_name_lenght'].isnull()

products_df[nulls_in_product_name_lenght]

In [19]:
for df_name in dataframe_names:
    df = globals()[df_name]
    null_counts = df.isnull().sum()
    null_cols = null_counts[null_counts > 0]
    if not null_cols.empty:
        print(f"\nDataFrame: {df_name}")
        for col, count in null_cols.items():
            print(f"  Coluna '{col}' tem {count} valores nulos.")
    else:
        print(f"\nDataFrame: {df_name} - Nenhum valor nulo encontrado.")



DataFrame: sellers_df - Nenhum valor nulo encontrado.

DataFrame: product_category_name_translation - Nenhum valor nulo encontrado.

DataFrame: orders_df
  Coluna 'order_approved_at' tem 160 valores nulos.
  Coluna 'order_delivered_carrier_date' tem 1783 valores nulos.
  Coluna 'order_delivered_customer_date' tem 2965 valores nulos.

DataFrame: order_items_df - Nenhum valor nulo encontrado.

DataFrame: customers_df - Nenhum valor nulo encontrado.

DataFrame: geolocation_df - Nenhum valor nulo encontrado.

DataFrame: order_payments_df - Nenhum valor nulo encontrado.

DataFrame: order_reviews_df
  Coluna 'review_comment_title' tem 87656 valores nulos.
  Coluna 'review_comment_message' tem 58247 valores nulos.

DataFrame: products_df
  Coluna 'product_category_name' tem 610 valores nulos.
  Coluna 'product_name_lenght' tem 610 valores nulos.
  Coluna 'product_description_lenght' tem 610 valores nulos.
  Coluna 'product_photos_qty' tem 610 valores nulos.
  Coluna 'product_weight_g' tem 2 

In [ ]:
for df_name in dataframe_names:
    df = globals()[df_name]

    if df.duplicated().any():
        print(f"DataFrame '{df_name}' tem duplicatas. Removendo...")
        df = df.drop_duplicates()
        globals()[df_name] = df

In [ ]:
geolocation_df#.head(3)
# geolocation_df.isnull().sum()

In [ ]:
order_items_df.head(3)

In [ ]:
order_payments_df.head(3)


In [ ]:
order_reviews_df.head(3)

In [ ]:
orders_df.head(3)

In [ ]:
products_df.head(3)

In [ ]:
sellers_df.head(3)

In [ ]:
product_category_name_translation.head(5)